# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902006


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:07<01:44,  3.72s/it]

Rendering models:  10%|█         | 3/30 [00:07<01:12,  2.70s/it]

Rendering models:  13%|█▎        | 4/30 [00:08<00:51,  1.99s/it]

Rendering models:  17%|█▋        | 5/30 [00:08<00:38,  1.52s/it]

...

Rendering models:  83%|████████▎ | 25/30 [00:16<00:02,  2.44it/s]

Rendering models:  90%|█████████ | 27/30 [00:16<00:00,  3.17it/s]

Rendering models:  93%|█████████▎| 28/30 [00:17<00:00,  2.59it/s]

Rendering models:  97%|█████████▋| 29/30 [00:17<00:00,  2.47it/s]

Bertrand15                            0.002348
dianapenguinp                         0.000866
twagner                               0.001276
elizabeth_s                           0.000936
not-logged-in-75af55cf19be29c510aa    0.012429
iceman73                              0.002202
equidad1                              0.051837
Rubru                                 0.001094
wrxtminecrafter                       0.000842
not-logged-in-56a60aa97859c0e5dccc    0.001973
smiralexx                             0.001433
peakscience7                          0.008083
RaulE                                 0.000580
pangeli5                              0.000915
Lavadude                              0.012542
not-logged-in-8700ff8251e0704f745c    0.039793
JScience5                             0.004825
beckyrother                           0.002737
fanboy_deluxe                         0.001574
GrantCrooks                           0.003676
Jackson2802                           0.000838
McMaster.Sam 

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())